In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Box, Layout, Output, Label
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from scipy import signal
from scipy.integrate import solve_ivp
from IPython.display import display, Latex, Markdown, Image, Math

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

# Stellgrößenbeschränkung

In [ ]:
imag = Image("../images/saturation.png", width=650)

outL = Output()
outR = Output()

with outL:
    display(imag)
with outR:
    display(Markdown("""
- lineares System erster Ordnung mit Störung $\\Delta u$ im Eingang
  \\begin{align*}
      \\dot{y}(t) & = a y(t) + b (u(t) + \\Delta u)
  \\end{align*}
- Steuerung mit $y_{\\mathrm{ref}} = $konst.
  \\begin{align*}
      u_{\\mathrm{ref}} & = \\frac{a}{b} y_{\\mathrm{ref}}
  \\end{align*}
- Begrenzung des Eingangs
  \\begin{align*}
      \\underline{u} & \\le u(t) \\le \\overline{u}
  \\end{align*}
- Regler: 
    - Verstärkungen: $K_{\\mathrm{P}} = l + k$, $K_{\\mathrm{I}} = l(b\\ k - a)$
    - Standard-PI-Regler mit und ohne Integratorstoppung
      \\begin{align*}
          u(t) & = u_{\\mathrm{ref}} - K_{\\mathrm{P}} \\tilde{y}(t) - K_{\\mathrm{I}} \\int_{t_0}^{t} \\tilde{y}(\\tau) \\mathrm{d} \\tau
      \\end{align*}
    - Beobachterformulierung:
      \\begin{align*}
          u(t) & = u_{\\mathrm{ref}} - K_{\\mathrm{P}} \\tilde{y}(t) - \\tilde{z}(t) \\\\
          \\dot{\\tilde{z}}(t) & = - \\frac{K_{\\mathrm{I}}}{K_{\\mathrm{P}}} (y(t) - y_{\\mathrm{ref}})
      \\end{align*}
    - Beobachterformulierung ohne Umrechnung
      \\begin{align*}
          u(t) & = u_{\\mathrm{ref}} - K_{\\mathrm{P}} \\tilde{y}(t) - \\tilde{z}(t) \\\\
          \\dot{\\tilde{z}}(t) & = - l b z(t) - l (l b + a) (y - y_{\\mathrm{ref}}) - l b (u(t) - u_{\\mathrm{ref}})
      \\end{align*}
"""))
cols = HBox([outL, outR], layout=Layout(display='flex', flex_flow='row', justify_content='space-around', align_items='center'))
display(cols)

**Definition Parameter**

In [ ]:
tSim = np.linspace(0, 5, 5001)

In [ ]:
output = widgets.Output()

with output:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True, gridspec_kw={'height_ratios': [1, 1]})

plt.subplots_adjust(wspace=0.2, hspace=0.07)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1.set_xlim([0, tSim[-1]])
ax2.set_xlim([0, tSim[-1]])
ax1.set_ylim([-0.01, 0.40]) 
ax2.set_ylim([-0.25, 0.25]) 
ax1.grid()
ax2.grid()
ax1.set_ylabel(r"$y$")
ax2.set_ylabel(r"$z$")
ax2.set_xlabel(r"$t$ in s")

yWoSys, = ax1.plot([], [], 'C0', label="PI-Regler ohne AW")
yWiSys, = ax1.plot([], [], 'C1--', label="PI-Regler mit AW")
yObsSys, = ax1.plot([], [], 'C2-.', label="red. Beobachter")
yObsAdaptSys, = ax1.plot([], [], 'C3:', label="adapt. red. Beobachter")
iWoSys, = ax2.plot([], [], 'C0')
iWiSys, = ax2.plot([], [], 'C1--')
iObsSys, = ax2.plot([], [], 'C2-.')
iObsAdaptSys, = ax2.plot([], [], 'C3:')

handlesAx, labelsAx = ax1.get_legend_handles_labels()

fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.94, 0.7735, .15), loc=3,
           ncol=4, mode="expand", borderaxespad=0., framealpha=0.5)

slidera = widgets.FloatSlider(value=-1,
                               min=-10,
                               max=-1,
                               step=0.5,
                               description=r'$a$')
sliderb = widgets.FloatSlider(value=1,
                               min=1,
                               max=10,
                               step=0.5,
                               description=r'$b$')
sliderk = widgets.FloatSlider(value=50,
                               min=10,
                               max=100,
                               step=1,
                               description=r'$k$')
sliderl = widgets.FloatSlider(value=25,
                               min=10,
                               max=100,
                               step=1,
                               description=r'$l$')
sliderdu = widgets.FloatSlider(value=1,
                               min=0,
                               max=5,
                               step=1,
                               description=r'$\Delta u$')
slideryr = widgets.FloatSlider(value=1,
                               min=0,
                               max=5,
                               step=1,
                               description=r'$y_{\mathrm{ref}}$')
sliderumax = widgets.FloatSlider(value=2,
                                 min=0,
                                 max=5,
                                 step=1,
                                 description=r'$u_{\mathrm{max}}$')

def sysCtrl(t, x, a, b, yref, uref, du, k, l, umax, method):
    y = x[0]
    z = x[1]
    dx = np.zeros(2)

    Kp = l + k
    Ki = l * ( b * k - a)
    
    if method == 0:
        u = - Ki * z - Kp * (y - yref)
        dx[1] = y - yref
    elif method == 1:
        u = -Ki * z - Kp * (y - yref)
        dx[1] = y - yref
        if abs(u) >= umax:
            dx[1] = 0
    elif method == 2:
        u = uref - Kp * (y - yref) - z
        u = min(umax, max(-umax, u))
        dx[1] = -Ki / Kp * (z + (u - uref))
    elif method == 3:
        u = uref - (k + l) * (y - yref) - z
        u = min(umax, max(-umax, u))
        dx[1] = -l * b * z - l * (l * b + a) * (y - yref) - l * b * (u - uref)

    u = min(umax, max(-umax, u))
    dx[0] = a * y + b * (u + du(t))
    return dx

def calcSystem(_):
    a = slidera.value
    b = sliderb.value
    k = sliderk.value
    l = sliderl.value
    du = sliderdu.value
    yr = slideryr.value
    umax = sliderumax.value

    ur = a / b * yr
    deltaU = lambda t: du if t < 2.0 else 0.0

    x0 = [0, 0]
    res = []
    for i in range(4):
        res.append(solve_ivp(sysCtrl,
                             [tSim[0], tSim[-1]],
                             x0,
                             t_eval=tSim, 
                             args=(a, b, yr, ur, deltaU, k, l, umax, i)))

    yWoSys.set_data(tSim, res[0].y.T[:, 0])
    yWiSys.set_data(tSim, res[1].y.T[:, 0])
    yObsSys.set_data(tSim, res[2].y.T[:, 0])
    yObsAdaptSys.set_data(tSim, res[3].y.T[:, 0])
    iWoSys.set_data(tSim, res[0].y.T[:, 1])
    iWiSys.set_data(tSim, res[1].y.T[:, 1])
    iObsSys.set_data(tSim, res[2].y.T[:, 1])
    iObsAdaptSys.set_data(tSim, res[3].y.T[:, 1])

    yMin = np.min([res[0].y.T[:, 0], res[1].y.T[:, 0], res[2].y.T[:, 0], res[3].y.T[:, 0]])
    yMax = np.max([res[0].y.T[:, 0], res[1].y.T[:, 0], res[2].y.T[:, 0], res[3].y.T[:, 0]])
    iMin = np.min([res[0].y.T[:, 1], res[1].y.T[:, 1], res[2].y.T[:, 1], res[3].y.T[:, 1]])
    iMax = np.max([res[0].y.T[:, 1], res[1].y.T[:, 1], res[2].y.T[:, 1], res[3].y.T[:, 1]])

    ax1.set_ylim(yMin - np.abs(yMax - yMin) * 0.1, yMax + np.abs(yMax - yMin) * 0.1)
    ax2.set_ylim(iMin - np.abs(iMax - iMin) * 0.1, iMax + np.abs(iMax - iMin) * 0.1)

    fig.canvas.draw()

slidera.observe(calcSystem, names='value')
sliderb.observe(calcSystem, names='value')
sliderdu.observe(calcSystem, names='value')
sliderk.observe(calcSystem, names='value')
sliderl.observe(calcSystem, names='value')
slideryr.observe(calcSystem, names='value')
sliderumax.observe(calcSystem, names='value')

calcSystem(_)

bModel = VBox([Label(value='Modell'), slidera, sliderb, sliderumax, sliderdu])
bModel.layout = make_box_layout()
bFeedBack = VBox([Label(value='Regler'), slideryr, sliderk, sliderl])
bFeedBack.layout = make_box_layout()

controls = HBox([bModel, bFeedBack])
VBox([controls, output], layout=Layout(display='flex', flex_flow='column', justify_content='center', align_items='center'))